In [8]:
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import warnings
#suppress warnings
warnings.filterwarnings('ignore')
print("Imports successful")

Imports successful


In [10]:
dataset = pd.read_csv(Path("../regressionAssignment/insurance_pre.csv"))
dataset = pd.get_dummies(dataset,drop_first=True)

independent_vars = ["age","bmi","children","sex_male","smoker_yes"]
dependent_var = "charges"

x = dataset[independent_vars]
y = dataset[dependent_var]
print(f"x.shape={x.shape} y.shape={y.shape}")



x.shape=(1338, 5) y.shape=(1338,)


In [11]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR

param_grid = {
    'kernel': ['rbf', 'poly', 'sigmoid', 'linear'],
    'C': [10, 100, 1000, 2000, 3000],
    'gamma': ['auto', 'scale']
}

#without scaling the code hangs
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_scaled = sc.fit_transform(x)


#refit if true will fit the model for best hyperparameter combination
grid = GridSearchCV(SVR(), param_grid, refit=True, verbose=3, n_jobs=-1)

# fitting the model for grid search
grid.fit(x_scaled, y)


Fitting 5 folds for each of 40 candidates, totalling 200 fits
[CV 1/5] END .....C=10, gamma=auto, kernel=poly;, score=0.037 total time=   0.0s
[CV 2/5] END .....C=10, gamma=auto, kernel=poly;, score=0.043 total time=   0.0s
[CV 5/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.048 total time=   0.0s
[CV 3/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.017 total time=   0.1s
[CV 1/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.050 total time=   0.0s
[CV 2/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.045 total time=   0.1s
[CV 3/5] END .....C=10, gamma=auto, kernel=poly;, score=0.066 total time=   0.1s
[CV 4/5] END .....C=10, gamma=auto, kernel=rbf;, score=-0.041 total time=   0.1s
[CV 5/5] END .....C=10, gamma=auto, kernel=poly;, score=0.034 total time=   0.0s
[CV 4/5] END .....C=10, gamma=auto, kernel=poly;, score=0.041 total time=   0.0s
[CV 3/5] END ...C=10, gamma=auto, kernel=linear;, score=0.485 total time=   0.0s
[CV 1/5] END ..C=10, gamma=auto, kernel=sigmoid

,"estimator estimator: estimator objectThis is assumed to implement the scikit-learn estimator interface.Either estimator needs to provide a ``score`` function,or ``scoring`` must be passed.",SVR()
,"param_grid param_grid: dict or list of dictionariesDictionary with parameters names (`str`) as keys and lists ofparameter settings to try as values, or a list of suchdictionaries, in which case the grids spanned by each dictionaryin the list are explored. This enables searching over any sequenceof parameter settings.","{'C': [10, 100, ...], 'gamma': ['auto', 'scale'], 'kernel': ['rbf', 'poly', ...]}"
,"scoring scoring: str, callable, list, tuple or dict, default=NoneStrategy to evaluate the performance of the cross-validated model onthe test set.If `scoring` represents a single score, one can use:- a single string (see :ref:`scoring_string_names`);- a callable (see :ref:`scoring_callable`) that returns a single value;- `None`, the `estimator`'s :ref:`default evaluation criterion ` is used.If `scoring` represents multiple scores, one can use:- a list or tuple of unique strings;- a callable returning a dictionary where the keys are the metric names and the values are the metric scores;- a dictionary with metric names as keys and callables as values.See :ref:`multimetric_grid_search` for an example.",None
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details... versionchanged:: v0.20 `n_jobs` default changed from 1 to None",-1
,"refit refit: bool, str, or callable, default=TrueRefit an estimator using the best found parameters on the wholedataset.For multiple metric evaluation, this needs to be a `str` denoting thescorer that would be used to find the best parameters for refittingthe estimator at the end.Where there are considerations other than maximum score inchoosing a best estimator, ``refit`` can be set to a function whichreturns the selected ``best_index_`` given ``cv_results_``. In thatcase, the ``best_estimator_`` and ``best_params_`` will be setaccording to the returned ``best_index_`` while the ``best_score_``attribute will not be available.The refitted estimator is made available at the ``best_estimator_``attribute and permits using ``predict`` directly on this``GridSearchCV`` instance.Also for multiple metric evaluation, the attributes ``best_index_``,``best_score_`` and ``best_params_`` will only be available if``refit`` is set and all of them will be determined w.r.t this specificscorer.See ``scoring`` parameter to know more about multiple metricevaluation.See :ref:`sphx_glr_auto_examples_model_selection_plot_grid_search_digits.py`to see how to design a custom selection strategy using a callablevia `refit`.See :ref:`this example`for an example of how to use ``refit=callable`` to balance modelcomplexity and cross-validated score... versionchanged:: 0.20 Support for callable added.",True
,"cv cv: int, cross-validation generator or an iterable, default=NoneDetermines the cross-validation splitting strategy.Possible inputs for cv are:- None, to use the default 5-fold cross validation,- integer, to specify the number of folds in a `(Stratified)KFold`,- :term:`CV splitter`,- An iterable yielding (train, test) splits as arrays of indices.For integer/None inputs, if the estimator is a classifier and ``y`` iseither binary or multiclass, :class:`StratifiedKFold` is used. In allother cases, :class:`KFold` is used. These splitters are instantiatedwith `shuffle=False` so the splits will be the same across calls.Refer :ref:`User Guide ` for the variouscross-validation strategies that can be used here... versionchanged:: 0.22 ``cv`` default value if None changed from 3-fold to 5-fold.",None
,"verbose verbose: intControls the verbosity: the higher, the more messages.- >1 : the computation time for each fold and parameter candidate is displayed;- >2 : the score is also displayed;- >3 

In [12]:
#print best parameter
print(f"Best parameter={grid.best_params_}")

Best parameter={'C': 3000, 'gamma': 'scale', 'kernel': 'rbf'}


In [13]:
#Let us see the results in tabular format
res_dict = grid.cv_results_
res_table = pd.DataFrame.from_dict(res_dict)

# Display only first 5 rows
print("First 5 rows:")
first_five_rows=res_table.head(5)
first_five_rows



First 5 rows:


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.036489,0.007280,0.019648,0.004210,10,auto,rbf,"{'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}",-0.050402,-0.045276,-0.016737,-0.041345,-0.048491,-0.040450,0.012248,36
1,0.031166,0.008201,0.005536,0.002726,10,auto,poly,"{'C': 10, 'gamma': 'auto', 'kernel': 'poly'}",0.037227,0.042884,0.065672,0.041406,0.034393,0.044316,0.011093,32
2,0.047927,0.007765,0.009064,0.003572,10,auto,sigmoid,"{'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}",0.029301,0.043379,0.060402,0.040689,0.017824,0.038319,0.014285,33
3,0.034180,0.011469,0.004257,0.002121,10,auto,linear,"{'C': 10, 'gamma': 'auto', 'kernel': 'linear'}",0.487873,0.472341,0.485102,0.475835,0.447692,0.473769,0.014239,25
4,0.034775,0.006789,0.015656,0.004656,10,scale,rbf,"{'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}",-0.050517,-0.045096,-0.016601,-0.041415,-0.048600,-0.040446,0.012321,35


In [ ]:
# Display the row that has rank_test_score = 1
# Note that hyper parameter combination 3000,scale,rbf is ranked no 1

print("\nRow with rank_test_score = 1:")
res_table[res_table['rank_test_score'] == 1]



Row with rank_test_score = 1:


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_gamma,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
36,0.062253,0.002143,0.017037,0.003707,3000,scale,rbf,"{'C': 3000, 'gamma': 'scale', 'kernel': 'rbf'}",0.87023,0.781912,0.855341,0.806008,0.823606,0.827419,0.032113,1


In [15]:
#Set this to True for Vscode and False for Jupyter notebook
use_widgets = True

if use_widgets:
  from ipywidgets import widgets
  from IPython.display import display

  age_widget = widgets.IntText(description='Age:', value=0)
  bmi_widget = widgets.FloatText(description='BMI:', value=0.0)
  children_widget = widgets.IntText(description='Children:', value=0)
  sex_widget = widgets.Checkbox(description='Male?', value=False)
  smoker_widget = widgets.Checkbox(description='Smoker?', value=False)

  display(age_widget, bmi_widget, children_widget, sex_widget, smoker_widget)

else:
  print("Widgets not available, using input() instead")
  age = int(input("Enter Age: "))
  bmi = float(input("Enter BMI: "))
  children = int(input("No. of children: "))
  sex_male = input("Male? (yes/no): ").lower() in ['yes', 'y', '1']
  smoker = input("Smoker? (yes/no): ").lower() in ['yes', 'y', '1']  




IntText(value=0, description='Age:')

FloatText(value=0.0, description='BMI:')

IntText(value=0, description='Children:')

Checkbox(value=False, description='Male?')

Checkbox(value=False, description='Smoker?')

In [16]:
if use_widgets:
  age = age_widget.value
  bmi = bmi_widget.value
  children = children_widget.value
  sex_male = sex_widget.value
  smoker = smoker_widget.value
  
print(f"age={age} bmi={bmi} children={children} male={sex_male} smoker={smoker}")

age=25 bmi=22.5 children=2 male=True smoker=False


In [17]:
#Let us do prediction
prediction = grid.predict([[age,bmi,children,sex_male,smoker]])
print(f"prediction={prediction}")

prediction=[17144.08720329]
